In [43]:
import pandas as pd
import json
from pprint import pprint
import random

In [32]:
file_path = './Clinical_and_Other_Features.xlsx'
data = pd.read_excel(file_path, sheet_name='Data')

# print("Columns in the dataset:")
# print(data.columns)

race_and_ethnicity_col = data.columns[21]
tumor_characteristics_col = data.columns[23:27]
mri_findings_col = data.columns[48:53]

# Print
# try:
#     print(data[mri_findings_col])
# except KeyError as e:
#     print(f"KeyError: {e}. Please check if the column names match exactly.")

In [34]:
first_row_descriptions = data.iloc[0]
print(first_row_descriptions)

Patient Information                                                 Patient ID
MRI Technical Information             Days to MRI (From the Date of Diagnosis)
Unnamed: 2                                                        Manufacturer
Unnamed: 3                                             Manufacturer Model Name
Unnamed: 4                                                        Scan Options
                                                   ...                        
Unnamed: 93                  Pathologic response to Neoadjuvant therapy:  P...
Unnamed: 94                  Pathologic response to Neoadjuvant therapy:  P...
Near Complete Response       Overall Near-complete Response:  Stricter Defi...
Unnamed: 96                  Overall Near-complete Response:  Looser Defini...
Unnamed: 97                            Near-complete Response (Graded Measure)
Name: 0, Length: 98, dtype: object


In [48]:
def row_to_vqa_json(row, patient_id, first_row, qid):
    questions = []
    
    # Race and Ethnicity
    race_questions_variants = [
        "What is the patient's race and ethnicity?",
        "Can you tell me the patient's race and ethnicity?",
        "Please provide the patient's race and ethnicity."
    ]
    race_question = {
        "Qid": qid,
        "Conversation": {
            "User": random.choice(race_questions_variants),
            "Gpt": row[race_and_ethnicity_col] 
        }
    }

    questions.append(race_question)

    vqa_json = {
        "Patient id": patient_id,
        "Questions": questions,
        #"Caption": {},
        "meta": {
            "Dataset": "Duke-Breast-Cancer-MRI",
            "Body location": "Breast",
        }
    }

    return vqa_json


In [51]:
vqa_json_list = []
global_qid = 1
for i in range(2, len(data)):
    patient_id = data.iloc[i][0]
    row = data.iloc[i]
    vqa_json = row_to_vqa_json(row, patient_id, first_row_descriptions, global_qid)
    global_qid += 1
    vqa_json_list.append(vqa_json)

[{'Patient id': 'Breast_MRI_001',
  'Questions': [{'Conversation': {'Gpt': 2,
                                  'User': "What is the patient's race and "
                                          'ethnicity?'},
                 'Qid': 1}],
  'meta': {'Body location': 'Breast', 'Dataset': 'Duke-Breast-Cancer-MRI'}},
 {'Patient id': 'Breast_MRI_002',
  'Questions': [{'Conversation': {'Gpt': 2,
                                  'User': "What is the patient's race and "
                                          'ethnicity?'},
                 'Qid': 2}],
  'meta': {'Body location': 'Breast', 'Dataset': 'Duke-Breast-Cancer-MRI'}},
 {'Patient id': 'Breast_MRI_003',
  'Questions': [{'Conversation': {'Gpt': 1,
                                  'User': "Please provide the patient's race "
                                          'and ethnicity.'},
                 'Qid': 3}],
  'meta': {'Body location': 'Breast', 'Dataset': 'Duke-Breast-Cancer-MRI'}},
 {'Patient id': 'Breast_MRI_004',
  'Quest

In [53]:
# pprint(vqa_json_list)
output_path = './duke_vqa.json'
with open(output_path, 'w') as f:
    json.dump(vqa_json_list, f, indent=4)
    print(f"Saved to {output_path}")

Saved to ./duke_vqa.json
